In [1]:
import os
import boto3
import pickle
import pandas as pd
from io import BytesIO
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# Configurações da AWS
AWS_ACCESS_KEY = user_secrets.get_secret("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = user_secrets.get_secret("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = user_secrets.get_secret("AWS_SESSION_TOKEN")
AWS_REGION = user_secrets.get_secret("AWS_REGION")
BUCKET_NAME = "datathon-base-458807800524"

# Criar conexão com o S3
def get_s3_client():
    return boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        aws_session_token=AWS_SESSION_TOKEN,
        region_name=AWS_REGION,
    )

def upload_to_s3(file_path, bucket_name=BUCKET_NAME, object_name=None):
    """Faz upload de um arquivo local para um bucket S3."""
    
    if object_name is None:
        object_name = os.path.basename(file_path)  # Nome padrão do arquivo no S3

    try:
        s3_client = get_s3_client()

        s3_client.upload_file(file_path, bucket_name, object_name)
        print(f"✔ Upload de {file_path} para s3://{bucket_name}/{object_name} concluído.")
        return True

    except FileNotFoundError:
        print("❌ O arquivo não foi encontrado.")
        return False
    except NoCredentialsError:
        print("❌ Credenciais da AWS não encontradas.")
        return False

In [3]:
def concat_dfs(path, low_memory = True):

    df_full = pd.DataFrame()
    
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            print('reading ', os.path.join(dirname, filename))
            df_auxiliar = pd.read_csv(os.path.join(dirname, filename), low_memory = False)
            df_full = pd.concat([df_full, df_auxiliar])

    return df_full

In [5]:
def update_files_folder(path, path_bucket, explode=False):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
                
            print('reading ', os.path.join(dirname, filename))
            
            upload_to_s3(
                file_path=os.path.join(dirname, filename),
                bucket_name=BUCKET_NAME,
                object_name=os.path.join(path_bucket, filename)
            )

In [6]:
def clear_news(df_news):
    df_news['issued'] = pd.to_datetime(df_news['issued'])
    df_news['modified'] = pd.to_datetime(df_news['modified'])

    return df_news
    

In [7]:
def clear_interactions(df_interactions):
    df_interactions = df_interactions.assign(
        history=df_interactions["history"].str.split(", "),
        timestampHistory=df_interactions["timestampHistory"].str.split(", "),
        numberOfClicksHistory=df_interactions["numberOfClicksHistory"].str.split(", "),
        timeOnPageHistory=df_interactions["timeOnPageHistory"].str.split(", "),
        scrollPercentageHistory=df_interactions["scrollPercentageHistory"].str.split(", "),
        pageVisitsCountHistory=df_interactions["pageVisitsCountHistory"].str.split(", "),
        timestampHistory_new=df_interactions["timestampHistory_new"].str.split(", ")
    ).explode([
        "history",
        "timestampHistory",
        "numberOfClicksHistory",
        "timeOnPageHistory",
        "scrollPercentageHistory",
        "pageVisitsCountHistory",
        "timestampHistory_new",
    ])

    # Garantir que a coluna é do tipo datetime, tratando timestamps em milissegundos
    df_interactions['timestampHistory'] = pd.to_numeric(df_interactions['timestampHistory'], errors='coerce')  # Converter para numérico, ignorando erros
    df_interactions['timestampHistory'] = pd.to_datetime(df_interactions['timestampHistory'], unit='ms', errors='coerce')  # Converter de timestamp Unix
    
    df_interactions['timestampHistory_new'] = pd.to_numeric(df_interactions['timestampHistory_new'], errors='coerce')  # Converter para numérico, ignorando erros
    df_interactions['timestampHistory_new'] = pd.to_datetime(df_interactions['timestampHistory_new'], unit='ms', errors='coerce')  # Converter de timestamp Unix


    return df_interactions

In [8]:
def save_dataframe_by_month(df, date_column, output_dir):
    """
    Salva um DataFrame separando os dados em arquivos por mês, baseado na coluna de data especificada.
    
    :param df: DataFrame do Pandas.
    :param date_column: Nome da coluna que contém as datas.
    :param output_dir: Diretório onde os arquivos CSV serão salvos.
    """
    if date_column not in df.columns:
        raise ValueError(f"A coluna '{date_column}' não está presente no DataFrame.")
    
    # Garantir que a coluna é do tipo datetime
    df[date_column] = pd.to_datetime(df[date_column])
    
    # Criar diretório de saída se não existir
    os.makedirs(output_dir, exist_ok=True)
    
    # Agrupar por ano e mês e salvar em arquivos separados
    for (year, month), group in df.groupby([df[date_column].dt.year, df[date_column].dt.month]):
        filename = os.path.join(output_dir, f"dados_{year}_{month:02d}.csv")
        group.to_csv(filename, index=False)
        print(f"Arquivo salvo: {filename}")

In [9]:
df_interactions = concat_dfs('/kaggle/input/dataton-pos-tech/files/treino')
df_news = concat_dfs('/kaggle/input/dataton-pos-tech/itens/itens')

reading  /kaggle/input/dataton-pos-tech/files/treino/treino_parte6.csv
reading  /kaggle/input/dataton-pos-tech/files/treino/treino_parte5.csv
reading  /kaggle/input/dataton-pos-tech/files/treino/treino_parte1.csv
reading  /kaggle/input/dataton-pos-tech/files/treino/treino_parte3.csv
reading  /kaggle/input/dataton-pos-tech/files/treino/treino_parte2.csv
reading  /kaggle/input/dataton-pos-tech/files/treino/treino_parte4.csv
reading  /kaggle/input/dataton-pos-tech/itens/itens/itens-parte2.csv
reading  /kaggle/input/dataton-pos-tech/itens/itens/itens-parte3.csv
reading  /kaggle/input/dataton-pos-tech/itens/itens/itens-parte1.csv


In [10]:
df_interactions = clear_interactions(df_interactions)
df_news = clear_news(df_news)

In [11]:
save_dataframe_by_month(df_news, 'modified', '/kaggle/working/noticias/')

Arquivo salvo: /kaggle/working/noticias/dados_2014_07.csv
Arquivo salvo: /kaggle/working/noticias/dados_2015_12.csv
Arquivo salvo: /kaggle/working/noticias/dados_2016_10.csv
Arquivo salvo: /kaggle/working/noticias/dados_2016_11.csv
Arquivo salvo: /kaggle/working/noticias/dados_2016_12.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_01.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_02.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_03.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_04.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_05.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_06.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_07.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_08.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_09.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_10.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_11.csv
Arquivo salvo: /kaggle/working/noticias/dados_2017_12.csv
Arquivo salvo:

In [12]:
save_dataframe_by_month(df_interactions, 'timestampHistory', '/kaggle/working/interactions/')

Arquivo salvo: /kaggle/working/interactions/dados_2022_07.csv
Arquivo salvo: /kaggle/working/interactions/dados_2022_08.csv


In [13]:
update_files_folder("/kaggle/working/noticias", "base_original/noticias")

reading  /kaggle/working/noticias/dados_2022_07.csv
✔ Upload de /kaggle/working/noticias/dados_2022_07.csv para s3://datathon-base-458807800524/base_original/noticias/dados_2022_07.csv concluído.
reading  /kaggle/working/noticias/dados_2018_11.csv
✔ Upload de /kaggle/working/noticias/dados_2018_11.csv para s3://datathon-base-458807800524/base_original/noticias/dados_2018_11.csv concluído.
reading  /kaggle/working/noticias/dados_2018_08.csv
✔ Upload de /kaggle/working/noticias/dados_2018_08.csv para s3://datathon-base-458807800524/base_original/noticias/dados_2018_08.csv concluído.
reading  /kaggle/working/noticias/dados_2020_10.csv
✔ Upload de /kaggle/working/noticias/dados_2020_10.csv para s3://datathon-base-458807800524/base_original/noticias/dados_2020_10.csv concluído.
reading  /kaggle/working/noticias/dados_2021_11.csv
✔ Upload de /kaggle/working/noticias/dados_2021_11.csv para s3://datathon-base-458807800524/base_original/noticias/dados_2021_11.csv concluído.
reading  /kaggle/wor

In [14]:
update_files_folder("/kaggle/working/interactions", "base_original/interacoes", True)

reading  /kaggle/working/interactions/dados_2022_07.csv
✔ Upload de /kaggle/working/interactions/dados_2022_07.csv para s3://datathon-base-458807800524/base_original/interacoes/dados_2022_07.csv concluído.
reading  /kaggle/working/interactions/dados_2022_08.csv
✔ Upload de /kaggle/working/interactions/dados_2022_08.csv para s3://datathon-base-458807800524/base_original/interacoes/dados_2022_08.csv concluído.
